In [6]:
import numpy as np
import pandas as pd

In [2]:
N = 600

t = np.arange(0, N, 1).reshape(-1,1)
t = np.array([t[i] + np.random.rand(1)/4 for i in range(len(t))])
t = np.array([t[i] - np.random.rand(1)/7 for i in range(len(t))])
t = np.array(np.round(t, 2))

x1 = np.round((np.random.random(N) * 5).reshape(-1,1), 2)
x2 = np.round((np.random.random(N) * 5).reshape(-1,1), 2)
x3 = np.round((np.random.random(N) * 5).reshape(-1,1), 2)

n = np.round((np.random.random(N) * 2).reshape(-1,1), 2)

y = np.array([((np.log(np.abs(2 + x1[t])) - x2[t-1]**2) + 0.02*x3[t-3]*np.exp(x1[t-1])) for t in range(len(t))])
y = np.round(y+n, 2)

In [4]:
n

array([[1.  ],
       [1.75],
       [1.11],
       [1.16],
       [1.84],
       [0.3 ],
       [1.62],
       [0.36],
       [0.57],
       [1.41],
       [0.09],
       [1.81],
       [1.12],
       [1.09],
       [1.84],
       [1.79],
       [1.05],
       [1.79],
       [0.61],
       [0.95],
       [1.01],
       [1.2 ],
       [0.5 ],
       [1.58],
       [1.03],
       [0.02],
       [1.1 ],
       [1.64],
       [0.33],
       [1.64],
       [1.39],
       [0.74],
       [1.37],
       [1.59],
       [0.31],
       [1.55],
       [0.25],
       [0.95],
       [0.99],
       [0.41],
       [1.78],
       [1.89],
       [0.56],
       [1.98],
       [1.71],
       [1.04],
       [0.26],
       [0.57],
       [1.51],
       [0.89],
       [0.16],
       [0.14],
       [0.54],
       [1.9 ],
       [1.18],
       [1.48],
       [0.08],
       [1.56],
       [1.77],
       [0.38],
       [0.46],
       [0.41],
       [0.07],
       [0.4 ],
       [1.65],
       [1.24],
       [0.

In [8]:
dataset = pd.DataFrame(np.concatenate((t, x1, x2, x3, y), axis=1), 
                       columns=['t', 'x1', 'x2', 'x3', 'y'])

deltaT = np.array([(dataset.t[i + 1] - dataset.t[i]) for i in range(len(dataset)-1)])
deltaT = np.concatenate((np.array([0]), deltaT))

dataset.insert(1, '∆t', deltaT)
dataset.head(10)

,t,∆t,x1,x2,x3,y
0,0.02,0.00,1.19,0.47,4.64,-1.53
1,1.09,1.07,4.53,3.07,2.05,3.52
2,1.97,0.88,3.97,1.96,1.34,0.63
3,3.12,1.15,4.11,3.93,4.88,4.05
4,4.20,1.08,3.12,0.72,1.25,-9.47
5,4.99,0.79,0.11,4.18,2.65,1.14
6,5.92,0.93,4.41,3.30,1.14,-13.89
7,7.08,1.16,1.03,2.86,3.47,-7.36
8,8.11,1.03,2.58,0.01,2.88,-5.94
9,9.00,0.89,2.45,1.58,0.35,3.20
